In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [2]:
spark = SparkSession.builder.appName('intro').getOrCreate()

In [3]:
spark

# chapter 2 .intro spark

In [4]:
my_range = spark.range(1000).toDF("number")

In [5]:
my_range.take(3)

[Row(number=0), Row(number=1), Row(number=2)]

In [7]:
my_range.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [8]:
divi_by2 = my_range.where('number %2 =0')

In [9]:
divi_by2.collect()

[Row(number=0),
 Row(number=2),
 Row(number=4),
 Row(number=6),
 Row(number=8),
 Row(number=10),
 Row(number=12),
 Row(number=14),
 Row(number=16),
 Row(number=18),
 Row(number=20),
 Row(number=22),
 Row(number=24),
 Row(number=26),
 Row(number=28),
 Row(number=30),
 Row(number=32),
 Row(number=34),
 Row(number=36),
 Row(number=38),
 Row(number=40),
 Row(number=42),
 Row(number=44),
 Row(number=46),
 Row(number=48),
 Row(number=50),
 Row(number=52),
 Row(number=54),
 Row(number=56),
 Row(number=58),
 Row(number=60),
 Row(number=62),
 Row(number=64),
 Row(number=66),
 Row(number=68),
 Row(number=70),
 Row(number=72),
 Row(number=74),
 Row(number=76),
 Row(number=78),
 Row(number=80),
 Row(number=82),
 Row(number=84),
 Row(number=86),
 Row(number=88),
 Row(number=90),
 Row(number=92),
 Row(number=94),
 Row(number=96),
 Row(number=98),
 Row(number=100),
 Row(number=102),
 Row(number=104),
 Row(number=106),
 Row(number=108),
 Row(number=110),
 Row(number=112),
 Row(number=114),
 Row(number

In [10]:
## end to end
flight_data_path = "data/flight-data/csv/2015-summary.csv"
flight_data_2015 = spark.read.options(inferschema=True,header=True).csv(flight_data_path)



In [11]:
flight_data_2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [12]:
flight_data_2015.sort('count').explain()

== Physical Plan ==
*(1) Sort [count#27 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#27 ASC NULLS FIRST, 200), true, [id=#80]
   +- FileScan csv [DEST_COUNTRY_NAME#25,ORIGIN_COUNTRY_NAME#26,count#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/c:/Users/Avinash Godbole/Desktop/Programm File/pyspark/oriily_book2018/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [15]:
# create view for sql queries
flight_data_2015.createOrReplaceTempView('flight_data')

In [16]:
dataFrameWay = flight_data_2015.groupBy('DEST_COUNTRY_NAME').count()

In [17]:
sqlway = spark.sql("select DEST_COUNTRY_NAME,count(1) from flight_data GROUP BY DEST_COUNTRY_NAME")

In [18]:
sqlway.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#25], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#25, 200), true, [id=#106]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#25], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#25] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/c:/Users/Avinash Godbole/Desktop/Programm File/pyspark/oriily_book2018/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [19]:
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#25], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#25, 200), true, [id=#125]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#25], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#25] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/c:/Users/Avinash Godbole/Desktop/Programm File/pyspark/oriily_book2018/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [20]:
flight_data_2015.select(F.max("count")).take(1)

[Row(max(count)=370002)]

In [21]:
# top 5 
# spark sql
max_sql = spark.sql("select DEST_COUNTRY_NAME, sum(count) as destination_total \
    from flight_data group by DEST_COUNTRY_NAME order by sum(count) desc limit 5")

In [27]:
max_sql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [31]:
# dataframe
flight_data_2015.groupBy("DEST_COUNTRY_NAME")\
    .sum('count')\
    .withColumnRenamed("sum(count)",'destination_total')\
    .sort(F.desc('destination_total'))\
    .limit(5)\
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [32]:
flight_data_2015.groupBy("DEST_COUNTRY_NAME")\
    .sum('count')\
    .withColumnRenamed("sum(count)",'destination_total')\
    .sort(F.desc('destination_total'))\
    .limit(5).explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#151L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#20,destination_total#151L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#20], functions=[sum(cast(count#22 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#20, 200), true, [id=#252]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#20], functions=[partial_sum(cast(count#22 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#20,count#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/c:/Users/Avinash Godbole/Desktop/Programm File/pyspark/oriily_book2018/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




# chapter 3 A Tour of Sparks Toolset

In [13]:
retail_data_path = 'data/retail-data/by-day/*.csv'
static_dataframe = spark.read.format("csv")\
    .options(inferschema=True,header=True)\
    .load(retail_data_path)

In [14]:
static_dataframe.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
static_dataframe.selectExpr(
    "CustomerID",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
    .groupBy(F.col("CustomerID"),F.window(F.col("InvoiceDate"),"1 day"))\
    .sum('total_cost')\
    .show()

+----------+--------------------+------------------+
|CustomerID|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   16057.0|[2011-12-05 05:30...|             -37.6|
|   14126.0|[2011-11-29 05:30...| 643.6300000000001|
|   13500.0|[2011-11-16 05:30...| 497.9700000000001|
|   17160.0|[2011-11-08 05:30...| 516.8499999999999|
|   15608.0|[2011-11-11 05:30...|             122.4|
|   15253.0|[2011-11-23 05:30...|             277.6|
|   15124.0|[2011-11-17 05:30...|             93.44|
|   12539.0|[2011-11-17 05:30...|           1050.66|
|   13658.0|[2011-11-30 05:30...| 542.4000000000001|
|   17396.0|[2011-10-31 05:30...|             495.0|
|   13576.0|[2011-11-10 05:30...| 543.3600000000001|
|   15111.0|[2011-11-10 05:30...|329.67999999999995|
|   17419.0|[2011-10-06 05:30...|465.54999999999995|
|   15749.0|[2011-04-18 05:30...|-1462.500000000001|
|   15769.0|[2011-04-18 05:30...|122.03999999999999|
|   18219.0|[2011-04-18 05:30...|            2

In [5]:
# in Python
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

In [6]:
step4.collect()

[Row(sum(id)=2500000000000)]

In [7]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#8L)])
+- Exchange SinglePartition, true, [id=#66]
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
      +- *(6) Project [id#8L]
         +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
            :- *(3) Sort [id#8L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#8L, 200), true, [id=#50]
            :     +- *(2) Project [(id#0L * 5) AS id#8L]
            :        +- Exchange RoundRobinPartitioning(5), false, [id=#46]
            :           +- *(1) Range (2, 10000000, step=2, splits=8)
            +- *(5) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200), true, [id=#57]
                  +- Exchange RoundRobinPartitioning(6), false, [id=#56]
                     +- *(4) Range (2, 10000000, step=4, splits=8)




# 4. Structured API 
    - DataFrame
    - Dataset 
    - sql

In [22]:
spark

In [26]:
df = spark.range(50).toDF("number")
df.withColumn("addition",df.number+10).show()

+------+--------+
|number|addition|
+------+--------+
|     0|      10|
|     1|      11|
|     2|      12|
|     3|      13|
|     4|      14|
|     5|      15|
|     6|      16|
|     7|      17|
|     8|      18|
|     9|      19|
|    10|      20|
|    11|      21|
|    12|      22|
|    13|      23|
|    14|      24|
|    15|      25|
|    16|      26|
|    17|      27|
|    18|      28|
|    19|      29|
+------+--------+
only showing top 20 rows



In [27]:
# rows 
df.collect()


[Row(number=0),
 Row(number=1),
 Row(number=2),
 Row(number=3),
 Row(number=4),
 Row(number=5),
 Row(number=6),
 Row(number=7),
 Row(number=8),
 Row(number=9),
 Row(number=10),
 Row(number=11),
 Row(number=12),
 Row(number=13),
 Row(number=14),
 Row(number=15),
 Row(number=16),
 Row(number=17),
 Row(number=18),
 Row(number=19),
 Row(number=20),
 Row(number=21),
 Row(number=22),
 Row(number=23),
 Row(number=24),
 Row(number=25),
 Row(number=26),
 Row(number=27),
 Row(number=28),
 Row(number=29),
 Row(number=30),
 Row(number=31),
 Row(number=32),
 Row(number=33),
 Row(number=34),
 Row(number=35),
 Row(number=36),
 Row(number=37),
 Row(number=38),
 Row(number=39),
 Row(number=40),
 Row(number=41),
 Row(number=42),
 Row(number=43),
 Row(number=44),
 Row(number=45),
 Row(number=46),
 Row(number=47),
 Row(number=48),
 Row(number=49)]

In [ ]:
# datatype

In [ ]:
# execution of plan page no 63 

In [ ]:
spark.stop()